# 🔢 Modern Integration Methods in Machine Learning

## From Theory to Industrial Applications

---

**Authored by:** AI-Mastery-2026 Project  
**Based on:** Draft by Marc Peter Deisenroth (UCL)  
**Date:** January 2026

### 🎯 Learning Objectives

By the end of this notebook, you will:

1. **Numerical Quadrature** - Master Newton-Cotes and Gaussian quadrature methods
2. **Monte Carlo Integration** - Understand random sampling with variance reduction
3. **Bayesian Quadrature** - Learn GP-based integration for expensive functions
4. **Normalizing Flows** - Transform simple distributions into complex ones
5. **Time Series Inference** - Apply EKF, UKF, and particle filtering

---

### 📚 Prerequisites

- Basic Python and NumPy
- Probability distributions (Gaussian, expectations)
- Linear algebra (matrix operations, Jacobians)

---

In [ ]:
# ============================================
# IMPORTS AND SETUP
# ============================================

import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import quad
from scipy.stats import norm
from scipy.special import roots_hermite, roots_legendre
import warnings

# Configuration
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-whitegrid')
np.random.seed(42)
np.set_printoptions(precision=4, suppress=True)

print("✅ Setup complete! NumPy version:", np.__version__)

---

# Introduction: Why Integration Matters in ML

---

Integration is fundamental to machine learning. Almost every probabilistic quantity we care about is an **expectation**:

$$\mathbb{E}_{x\sim p}[f(x)] := \int f(x)p(x)dx$$

### Key Applications:

| Application | Integral Form | ML Example |
|-------------|---------------|------------|
| **Mean** | $\mathbb{E}[x] = \int xp(x)dx$ | Predicted value |
| **Variance** | $\mathbb{V}[x] = \int (x-\mu)^2p(x)dx$ | Uncertainty |
| **Marginal Likelihood** | $p(Y|X) = \int p(Y|X,\theta)p(\theta)d\theta$ | Model selection |
| **Bayesian Prediction** | $p(y^*|x^*,D) = \int p(y^*|\theta)p(\theta|D)d\theta$ | Predictive uncertainty |

### 🏭 Industrial Impact:

- **JPMorgan Chase**: Risk assessment using Monte Carlo for Value-at-Risk
- **DeepMind Health**: Uncertainty quantification in medical diagnosis
- **Tesla Autopilot**: Bayesian sensor fusion for trajectory prediction
- **Netflix**: Recommendation under uncertainty (200M+ users)

---

# Chapter 1: Newton-Cotes Methods

---

Newton-Cotes methods approximate integrals using polynomial interpolation at equidistant points.

## 1.1 Trapezoidal Rule

### Theory

The trapezoidal rule approximates $f(x)$ as linear between consecutive points:

$$\int_a^b f(x)dx \approx \frac{h}{2}(f_0 + 2f_1 + \cdots + 2f_{N-1} + f_N)$$

where $h = (b-a)/N$ and $f_n = f(x_n)$ at equidistant nodes.

**Error**: $O(1/N^2)$ - suitable for non-smooth functions

### 🏭 Industrial Use Case: Boeing Aerodynamics

Boeing engineers use trapezoidal integration to compute lift coefficients by integrating pressure distributions over wing surfaces. The method's robustness to measurement noise makes it ideal for wind tunnel data.

### 📝 Interview Question

> **Q**: When would you prefer trapezoidal rule over Simpson's?
>
> **A**: For noisy data or non-smooth functions. Simpson's quadratic fit can amplify oscillations in noisy measurements, while trapezoidal's linear approximation is more stable.

In [ ]:
# ============================================
# 1.1 TRAPEZOIDAL RULE IMPLEMENTATION
# ============================================

def trapezoidal_rule(f, a, b, n):
    """
    Trapezoidal rule for numerical integration.
    
    Error: O(1/N²)
    """
    x = np.linspace(a, b, n + 1)
    y = f(x)
    h = (b - a) / n
    return h * (0.5 * y[0] + np.sum(y[1:-1]) + 0.5 * y[-1])


# Example: Integrate exp(-x² - sin(3x)²) from 0 to 1
def test_function(x):
    return np.exp(-x**2 - np.sin(3*x)**2)

# True value using high-precision quadrature
true_value, _ = quad(test_function, 0, 1, epsabs=1e-12)

# Test with different node counts
nodes = [5, 10, 20, 40, 80]
trap_results = [trapezoidal_rule(test_function, 0, 1, n) for n in nodes]
trap_errors = [abs(r - true_value) for r in trap_results]

print("="*50)
print("TRAPEZOIDAL RULE CONVERGENCE")
print("="*50)
print(f"True value: {true_value:.10f}")
print()
for n, result, error in zip(nodes, trap_results, trap_errors):
    print(f"N={n:3d}: Result={result:.10f}, Error={error:.2e}")

## 1.2 Simpson's Rule

### Theory

Simpson's rule uses **quadratic** interpolation (parabolas) connecting triplets of points:

$$\int_a^b f(x)dx \approx \frac{h}{3}(f_0 + 4f_1 + 2f_2 + 4f_3 + \cdots + 4f_{N-1} + f_N)$$

**Error**: $O(1/N^4)$ - much faster convergence for smooth functions!

### 🏭 Industrial Use Case: NASA JPL

NASA's Jet Propulsion Laboratory uses Simpson's rule in spacecraft trajectory calculations. The Cassini mission to Saturn used these techniques for fuel-efficient navigation through the solar system.

### 📝 Interview Question

> **Q**: Derive the error bound for Simpson's rule.
>
> **A**: For Simpson's: Error = $-\frac{(b-a)^5}{180N^4}f^{(4)}(\xi)$ for some $\xi \in [a,b]$. The fourth derivative term explains why it's exact for cubics.

In [ ]:
# ============================================
# 1.2 SIMPSON'S RULE IMPLEMENTATION
# ============================================

def simpsons_rule(f, a, b, n):
    """
    Simpson's 1/3 rule for numerical integration.
    
    Error: O(1/N⁴) - faster convergence for smooth functions
    """
    if n % 2 != 0:
        n += 1  # Ensure even
    
    x = np.linspace(a, b, n + 1)
    y = f(x)
    h = (b - a) / n
    
    result = y[0] + y[-1]
    result += 4 * np.sum(y[1:-1:2])  # Odd indices
    result += 2 * np.sum(y[2:-2:2])  # Even indices
    
    return h * result / 3


# Compare convergence
simp_results = [simpsons_rule(test_function, 0, 1, n) for n in nodes]
simp_errors = [abs(r - true_value) for r in simp_results]

print("="*50)
print("SIMPSON'S RULE CONVERGENCE")
print("="*50)
for n, result, error in zip(nodes, simp_results, simp_errors):
    print(f"N={n:3d}: Result={result:.10f}, Error={error:.2e}")

# Plot convergence comparison
plt.figure(figsize=(10, 6))
plt.loglog(nodes, trap_errors, 'o-', linewidth=2, label='Trapezoidal')
plt.loglog(nodes, simp_errors, 's-', linewidth=2, label="Simpson's")
plt.loglog(nodes, [trap_errors[0]*(nodes[0]/n)**2 for n in nodes], 
           'k--', alpha=0.5, label='$O(1/N^2)$')
plt.loglog(nodes, [simp_errors[0]*(nodes[0]/n)**4 for n in nodes], 
           'k-.', alpha=0.5, label='$O(1/N^4)$')
plt.xlabel('Number of nodes', fontsize=12)
plt.ylabel('Integration error', fontsize=12)
plt.title('Newton-Cotes Convergence Comparison', fontsize=14, fontweight='bold')
plt.legend(fontsize=10)
plt.grid(True, which="both", ls="-", alpha=0.3)
plt.tight_layout()
plt.show()

print("\n💡 Key Insight: Simpson's error decreases 16x when N doubles,")
print("   while Trapezoidal only decreases 4x.")

---

# Chapter 2: Gaussian Quadrature

---

Gaussian quadrature optimizes **both** nodes and weights to exactly integrate polynomials up to degree $2N-1$.

$$\int_a^b f(x)w(x)dx \approx \sum_{n=1}^N w_n f(x_n)$$

| Interval | Weight $w(x)$ | Orthogonal Polynomial |
|----------|---------------|----------------------|
| $[-1, 1]$ | $1$ | Legendre |
| $[-1, 1]$ | $(1-x^2)^{-1/2}$ | Chebyshev |
| $[0, \infty]$ | $e^{-x}$ | Laguerre |
| $(-\infty, \infty)$ | $e^{-x^2}$ | **Hermite** |

## 2.1 Gauss-Hermite Quadrature

### Theory

Gauss-Hermite is designed for integrals with Gaussian weight:

$$\int_{-\infty}^{\infty} f(x)e^{-x^2}dx \approx \sum_{n=1}^N w_n f(x_n)$$

For expectations under $\mathcal{N}(\mu, \sigma^2)$:

$$\mathbb{E}_{x\sim\mathcal{N}(\mu,\sigma^2)}[f(x)] \approx \frac{1}{\sqrt{\pi}}\sum_{n=1}^N w_n f(\sqrt{2}\sigma x_n + \mu)$$

### 🏭 Industrial Use Case: BlackRock Option Pricing

BlackRock uses Gauss-Hermite quadrature in option pricing models to compute expected payoffs under Gaussian market scenarios. With only 10-20 points, they achieve accuracy that would require 100,000+ Monte Carlo samples.

### 📝 Interview Question

> **Q**: How would you compute $\mathbb{E}[\text{ReLU}(X)]$ where $X \sim \mathcal{N}(0,1)$?
>
> **A**: Use Gauss-Hermite with $f(x) = \max(0, x)$. Transform nodes via $\sqrt{2}\sigma x_n + \mu$. The answer is $1/\sqrt{2\pi} \approx 0.399$.

In [ ]:
# ============================================
# 2.1 GAUSS-HERMITE QUADRATURE
# ============================================

def gauss_hermite_expectation(f, mu=0, sigma=1, n=10):
    """
    Compute E[f(X)] where X ~ N(μ, σ²) using Gauss-Hermite.
    
    This is the go-to method for Gaussian expectations!
    """
    nodes, weights = roots_hermite(n)
    
    # Transform for N(μ, σ²)
    transformed_nodes = np.sqrt(2) * sigma * nodes + mu
    transformed_weights = weights / np.sqrt(np.pi)
    
    return np.sum(transformed_weights * f(transformed_nodes))


# Test: E[X²] for X ~ N(0,1) should be 1
f_squared = lambda x: x**2
result = gauss_hermite_expectation(f_squared, mu=0, sigma=1, n=5)
print(f"E[X²] for X~N(0,1): {result:.10f} (should be 1.0)")

# Test: E[ReLU(X)] for X ~ N(0,1)
f_relu = lambda x: np.maximum(0, x)
relu_result = gauss_hermite_expectation(f_relu, mu=0, sigma=1, n=20)
true_relu = 1 / np.sqrt(2 * np.pi)  # Analytical result
print(f"E[ReLU(X)] for X~N(0,1): {relu_result:.6f} (analytical: {true_relu:.6f})")

# Visualize quadrature points
n_points = 10
nodes, weights = roots_hermite(n_points)
transformed_nodes = np.sqrt(2) * nodes

x = np.linspace(-4, 4, 500)
f = lambda x: np.exp(-x**2/2) * np.cos(3*x)

plt.figure(figsize=(12, 5))

# Left: Function and quadrature points
plt.subplot(1, 2, 1)
plt.plot(x, f(x) * norm.pdf(x), 'b-', linewidth=2, label='f(x)p(x)')
plt.fill_between(x, f(x) * norm.pdf(x), alpha=0.2)
plt.stem(transformed_nodes, f(transformed_nodes) * norm.pdf(transformed_nodes),
         'r', markerfmt='ro', basefmt=' ', label='Quadrature points')
plt.xlabel('x')
plt.ylabel('f(x)p(x)')
plt.title(f'Gauss-Hermite Quadrature (N={n_points})', fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)

# Right: Convergence
plt.subplot(1, 2, 2)
n_values = range(2, 25)
mc_estimate = np.mean(f(np.random.randn(100000)))  # MC reference
gh_estimates = [gauss_hermite_expectation(f, n=n) for n in n_values]
gh_errors = [abs(e - mc_estimate) for e in gh_estimates]

plt.semilogy(list(n_values), gh_errors, 'o-', linewidth=2)
plt.xlabel('Number of quadrature points')
plt.ylabel('Error (log scale)')
plt.title('Gauss-Hermite Convergence', fontweight='bold')
plt.grid(True, which='both', alpha=0.3)

plt.tight_layout()
plt.show()

print("\n🔑 Key Insight: Just 10 Gauss-Hermite points often beat 100,000 MC samples!")

---

# Chapter 3: Monte Carlo Integration

---

Monte Carlo integration uses **random sampling** to approximate expectations:

$$\mathbb{E}_{x\sim p}[f(x)] \approx \frac{1}{S}\sum_{s=1}^S f(x^{(s)}), \quad x^{(s)} \sim p(x)$$

### Key Properties:

- **Unbiased**: Expected value equals true integral
- **Convergence**: $O(1/\sqrt{S})$, **independent of dimension**!
- **Curse of dimensionality-free**: Works in 1000+ dimensions

### 🏭 Industrial Use Cases:

| Company | Application |
|---------|-------------|
| **Netflix** | Expected watch time for 200M users |
| **JPMorgan** | Value-at-Risk with 100K+ scenarios |
| **Pixar** | Global illumination (path tracing) |
| **CERN** | Particle physics cross-sections |

### 📝 Interview Question

> **Q**: Why does Monte Carlo work well in high dimensions when quadrature fails?
>
> **A**: Quadrature needs $O(N^d)$ points (curse of dimensionality). MC converges at $O(1/\sqrt{N})$ regardless of $d$. At $d > 10$, MC is almost always better.

In [ ]:
# ============================================
# 3.1 BASIC MONTE CARLO
# ============================================

def monte_carlo_integrate(f, sampler, n_samples, seed=None):
    """
    Monte Carlo integration: E_p[f(X)] ≈ (1/N) Σ f(x_i)
    
    Returns (estimate, standard_error)
    """
    if seed is not None:
        np.random.seed(seed)
    
    samples = sampler(n_samples)
    values = f(samples)
    
    estimate = np.mean(values)
    standard_error = np.std(values) / np.sqrt(n_samples)
    
    return estimate, standard_error


# Example: E[X²] for X ~ N(0,1)
f = lambda x: x**2
sampler = lambda n: np.random.randn(n)

sample_sizes = [100, 500, 1000, 5000, 10000, 50000, 100000]
estimates = []
errors = []

for n in sample_sizes:
    est, se = monte_carlo_integrate(f, sampler, n, seed=42)
    estimates.append(est)
    errors.append(se)

print("="*50)
print("MONTE CARLO CONVERGENCE")
print("="*50)
print(f"True value: E[X²] = 1.0")
print()
for n, est, se in zip(sample_sizes, estimates, errors):
    print(f"N={n:6d}: Estimate={est:.6f}, SE={se:.6f}")

# Plot convergence
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.errorbar(np.log10(sample_sizes), estimates, yerr=2*np.array(errors), 
             fmt='o-', capsize=3)
plt.axhline(y=1.0, color='r', linestyle='--', label='True value')
plt.xlabel('log₁₀(N)')
plt.ylabel('Estimate')
plt.title('Monte Carlo Estimates with 95% CI', fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.loglog(sample_sizes, errors, 'o-', linewidth=2, label='Standard Error')
plt.loglog(sample_sizes, [errors[0]*np.sqrt(sample_sizes[0]/n) for n in sample_sizes],
           'r--', label='O(1/√N)')
plt.xlabel('N')
plt.ylabel('Standard Error')
plt.title('Error Convergence Rate', fontweight='bold')
plt.legend()
plt.grid(True, which='both', alpha=0.3)

plt.tight_layout()
plt.show()

## 3.2 Importance Sampling

### Theory

When sampling from $p(x)$ is difficult or inefficient, use a **proposal** $q(x)$:

$$\mathbb{E}_p[f(x)] = \mathbb{E}_q\left[f(x)\frac{p(x)}{q(x)}\right] \approx \frac{1}{S}\sum_{s=1}^S w_s f(x^{(s)})$$

where $w_s = p(x^{(s)})/q(x^{(s)})$ are **importance weights**.

### 🏭 Industrial Use Case: JPMorgan Rare Events

JPMorgan uses importance sampling for estimating tail risks (e.g., probability of losses > $100M). Standard MC would rarely sample extreme events. By shifting the proposal toward the tail, they estimate rare event probabilities with orders of magnitude fewer samples.

### 📝 Interview Question

> **Q**: How do you choose a good proposal distribution?
>
> **A**: The optimal proposal minimizes variance: $q^*(x) \propto |f(x)|p(x)$. In practice, choose $q$ that is similar to $|f|p$ but easier to sample from. For rare events, shift $q$ toward the rare region.

In [ ]:
# ============================================
# 3.2 IMPORTANCE SAMPLING
# ============================================

def importance_sampling(f, p, q, q_sampler, n_samples, seed=None):
    """
    Importance sampling: E_p[f(X)] using samples from q.
    
    Returns (estimate, standard_error, effective_sample_size)
    """
    if seed is not None:
        np.random.seed(seed)
    
    samples = q_sampler(n_samples)
    
    # Importance weights
    weights = p(samples) / (q(samples) + 1e-10)
    normalized_weights = weights / np.sum(weights)
    
    # Weighted estimate
    f_vals = f(samples)
    estimate = np.sum(normalized_weights * f_vals)
    
    # Effective sample size
    ess = 1.0 / np.sum(normalized_weights**2)
    
    return estimate, ess


# Example: Estimate P(X > 3) for X ~ N(0,1)
# True value: ~0.00135

def indicator_above_3(x):
    return (x > 3).astype(float)

# Standard MC
p_standard = lambda x: norm.pdf(x, 0, 1)
q_standard = lambda x: norm.pdf(x, 0, 1) 
q_standard_sampler = lambda n: np.random.randn(n)

# Importance sampling with shifted proposal
q_shifted = lambda x: norm.pdf(x, 3.5, 1)  # Centered near the rare region
q_shifted_sampler = lambda n: np.random.randn(n) + 3.5

n_samples = 10000
true_prob = 1 - norm.cdf(3)  # ~0.00135

# Run multiple trials
n_trials = 50
mc_estimates = []
is_estimates = []

for trial in range(n_trials):
    # Standard MC
    samples = np.random.randn(n_samples)
    mc_estimates.append(np.mean(samples > 3))
    
    # Importance sampling
    is_est, _ = importance_sampling(
        indicator_above_3, p_standard, q_shifted, q_shifted_sampler, n_samples
    )
    is_estimates.append(is_est)

print("="*50)
print("IMPORTANCE SAMPLING: RARE EVENT ESTIMATION")
print("="*50)
print(f"True P(X > 3): {true_prob:.6f}")
print()
print(f"Standard MC:    Mean={np.mean(mc_estimates):.6f}, Std={np.std(mc_estimates):.6f}")
print(f"Importance Samp: Mean={np.mean(is_estimates):.6f}, Std={np.std(is_estimates):.6f}")
print()
print(f"Variance reduction: {np.var(mc_estimates)/np.var(is_estimates):.1f}x")

# Visualize
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
x = np.linspace(-2, 6, 500)
plt.plot(x, norm.pdf(x, 0, 1), 'b-', linewidth=2, label='Target p(x) = N(0,1)')
plt.plot(x, norm.pdf(x, 3.5, 1), 'r-', linewidth=2, label='Proposal q(x) = N(3.5,1)')
plt.axvline(x=3, color='green', linestyle='--', label='Threshold (x=3)')
plt.fill_between(x[x>3], norm.pdf(x[x>3], 0, 1), alpha=0.3, color='green')
plt.xlabel('x')
plt.ylabel('Density')
plt.title('Importance Sampling for Rare Events', fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.hist(mc_estimates, bins=20, alpha=0.5, label='Standard MC', density=True)
plt.hist(is_estimates, bins=20, alpha=0.5, label='Importance Sampling', density=True)
plt.axvline(x=true_prob, color='r', linestyle='--', linewidth=2, label='True value')
plt.xlabel('Estimate')
plt.ylabel('Density')
plt.title('Estimator Distributions (50 trials)', fontweight='bold')
plt.legend()

plt.tight_layout()
plt.show()

---

# Chapter 4: Normalizing Flows

---

Normalizing flows transform simple distributions into complex ones through **invertible** transformations.

### Theory

Given base distribution $p_0(z_0)$ and transformations $z_k = f_k(z_{k-1})$:

$$\log p(x) = \log p(z_0) - \sum_{k=1}^K \log\left|\det\frac{\partial f_k}{\partial z_{k-1}}\right|$$

The Jacobian determinant accounts for volume change under transformation.

### 🏭 Industrial Use Cases:

| Company | Application | Result |
|---------|-------------|--------|
| **Spotify** | User preference modeling | 23% engagement improvement |
| **Waymo** | Pedestrian trajectory prediction | Multi-modal futures |
| **NVIDIA** | Video synthesis | Realistic temporal dynamics |

### 📝 Interview Question

> **Q**: Compare normalizing flows to VAEs and GANs.
>
> **A**: Flows give **exact** likelihoods (no ELBO bound). They're invertible (unlike GANs). Trade-off: architecture restricted for tractable Jacobian ($O(D)$ not $O(D^3)$).

In [ ]:
# ============================================
# 4.1 PLANAR FLOW IMPLEMENTATION
# ============================================

class PlanarFlow:
    """
    Planar flow: f(z) = z + u * tanh(w^T z + b)
    
    Contracts/expands space along hyperplanes.
    """
    
    def __init__(self, d):
        self.d = d
        self.w = np.random.randn(d) * 0.5
        self.u = np.random.randn(d) * 0.5
        self.b = np.random.randn() * 0.5
        self._enforce_invertibility()
    
    def _enforce_invertibility(self):
        """Ensure w^T u >= -1 for invertibility."""
        wtu = np.dot(self.w, self.u)
        m_wtu = -1 + np.log(1 + np.exp(wtu))
        self.u_hat = self.u + (m_wtu - wtu) * self.w / (np.linalg.norm(self.w)**2 + 1e-8)
    
    def forward(self, z):
        """Transform z -> z_new, return (z_new, log_det_jacobian)"""
        z = np.atleast_2d(z)
        self._enforce_invertibility()
        
        linear = z @ self.w + self.b
        h = np.tanh(linear)
        z_new = z + np.outer(h, self.u_hat)
        
        h_prime = 1 - h**2
        psi = h_prime * np.dot(self.u_hat, self.w)
        log_det = np.log(np.abs(1 + psi) + 1e-8)
        
        return z_new, log_det


class FlowChain:
    """Chain of normalizing flows."""
    
    def __init__(self, flows):
        self.flows = flows
    
    def forward(self, z):
        z = np.atleast_2d(z)
        total_log_det = np.zeros(z.shape[0])
        
        for flow in self.flows:
            z, log_det = flow.forward(z)
            total_log_det += log_det
        
        return z, total_log_det


# Visualize flow transformation
np.random.seed(42)
n_samples = 2000
n_flows_list = [0, 2, 4, 8]

fig, axes = plt.subplots(1, 4, figsize=(16, 4))

for ax, n_flows in zip(axes, n_flows_list):
    z0 = np.random.randn(n_samples, 2)
    
    if n_flows == 0:
        z = z0
        title = 'Base Distribution'
    else:
        flows = [PlanarFlow(2) for _ in range(n_flows)]
        chain = FlowChain(flows)
        z, _ = chain.forward(z0)
        title = f'{n_flows} Planar Flows'
    
    ax.scatter(z[:, 0], z[:, 1], alpha=0.3, s=5)
    ax.set_xlim(-4, 4)
    ax.set_ylim(-4, 4)
    ax.set_title(title, fontweight='bold')
    ax.set_aspect('equal')
    ax.grid(True, alpha=0.3)

plt.suptitle('Normalizing Flow: Transforming Gaussian to Complex Distribution', 
             fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("💡 Key Insight: More flows = more expressive distribution.")
print("   Spotify uses 8-16 flows for user preference modeling.")

---

# Chapter 5: Time Series Inference

---

For state space models:

$$x_{t+1} = f(x_t) + \epsilon, \quad \epsilon \sim \mathcal{N}(0, Q)$$

We need to propagate uncertainty through nonlinear dynamics.

### Methods:

| Method | Approach | Complexity | Best For |
|--------|----------|------------|----------|
| **EKF** | Linearization | $O(D^3)$ | Mildly nonlinear |
| **UKF** | Sigma points | $O(D^3)$ | Moderate nonlinearity |
| **Particle Filter** | Sampling | $O(N)$ | Multi-modal, non-Gaussian |

### 🏭 Industrial Use Cases:

| Company | Method | Application |
|---------|--------|-------------|
| **Tesla FSD** | EKF + UKF | Vehicle trajectory prediction |
| **Boston Dynamics** | EKF | Robot state estimation |
| **Garmin** | Kalman Filter | GPS position tracking |

### 📝 Interview Question

> **Q**: When would you use a particle filter over EKF/UKF?
>
> **A**: For multi-modal posteriors or non-Gaussian noise. EKF/UKF assume Gaussian. Particle filters can represent arbitrary distributions but scale poorly with state dimension.

In [ ]:
# ============================================
# 5.1 UNSCENTED TRANSFORM (UKF Core)
# ============================================

def unscented_transform(f, mu, sigma, alpha=1e-3, beta=2, kappa=0):
    """
    Unscented transform for propagating Gaussian through nonlinear function.
    
    Uses 2D+1 sigma points to capture mean and covariance.
    """
    D = len(mu)
    lam = alpha**2 * (D + kappa) - D
    
    # Compute sigma points
    sqrt_matrix = np.linalg.cholesky((D + lam) * sigma)
    
    sigma_points = np.zeros((2*D + 1, D))
    sigma_points[0] = mu
    for i in range(D):
        sigma_points[i+1] = mu + sqrt_matrix[i]
        sigma_points[D+i+1] = mu - sqrt_matrix[i]
    
    # Weights
    Wm = np.full(2*D + 1, 1 / (2*(D + lam)))
    Wm[0] = lam / (D + lam)
    Wc = Wm.copy()
    Wc[0] += (1 - alpha**2 + beta)
    
    # Propagate through function
    Y = np.array([f(sp) for sp in sigma_points])
    
    # Compute output mean and covariance
    mu_y = np.sum(Wm[:, None] * Y, axis=0)
    sigma_y = np.zeros((Y.shape[1], Y.shape[1]))
    for i in range(2*D + 1):
        diff = Y[i] - mu_y
        sigma_y += Wc[i] * np.outer(diff, diff)
    
    return mu_y, sigma_y, sigma_points, Y


# Example: Propagate through nonlinear function
def nonlinear_dynamics(x):
    """Example nonlinear state transition."""
    return np.array([np.sin(x[0]) + 0.5*x[1], 
                     0.9*x[1] + 0.1*x[0]**2])

# Initial state
mu_0 = np.array([1.0, 0.5])
sigma_0 = np.array([[0.2, 0.05], [0.05, 0.1]])

# Apply unscented transform
mu_1, sigma_1, sigma_pts, Y = unscented_transform(nonlinear_dynamics, mu_0, sigma_0)

print("="*50)
print("UNSCENTED TRANSFORM EXAMPLE")
print("="*50)
print(f"Input mean: {mu_0}")
print(f"Input cov diagonal: {np.diag(sigma_0)}")
print()
print(f"Output mean: {mu_1}")
print(f"Output cov diagonal: {np.diag(sigma_1)}")
print()
print(f"Number of sigma points: {len(sigma_pts)} (2D+1 = {2*len(mu_0)+1})")

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Input space
ax = axes[0]
samples = np.random.multivariate_normal(mu_0, sigma_0, 500)
ax.scatter(samples[:, 0], samples[:, 1], alpha=0.3, s=10, label='Samples')
ax.scatter(sigma_pts[:, 0], sigma_pts[:, 1], c='red', s=100, marker='x', 
           linewidths=2, label='Sigma points')
ax.scatter(mu_0[0], mu_0[1], c='green', s=150, marker='*', label='Mean')
ax.set_xlabel('x₁')
ax.set_ylabel('x₂')
ax.set_title('Input Distribution', fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

# Output space
ax = axes[1]
samples_out = np.array([nonlinear_dynamics(s) for s in samples])
ax.scatter(samples_out[:, 0], samples_out[:, 1], alpha=0.3, s=10, label='Propagated samples')
ax.scatter(Y[:, 0], Y[:, 1], c='red', s=100, marker='x', 
           linewidths=2, label='Propagated sigma points')
ax.scatter(mu_1[0], mu_1[1], c='green', s=150, marker='*', label='UKF mean')
ax.set_xlabel('y₁')
ax.set_ylabel('y₂')
ax.set_title('Output Distribution (through nonlinear f)', fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n🔑 Key Insight: UKF uses 2D+1 points to capture mean/covariance exactly")
print("   for linear transforms, and approximates well for nonlinear.")

---

# Summary & Key Takeaways

---

## Methods Comparison

| Method | Convergence | Dimension | Best For |
|--------|-------------|-----------|----------|
| **Trapezoidal** | O(1/N²) | Low | Noisy data |
| **Simpson's** | O(1/N⁴) | Low | Smooth functions |
| **Gauss-Hermite** | Exponential | Low | Gaussian expectations |
| **Monte Carlo** | O(1/√N) | Any | High dimensions |
| **Importance Sampling** | Variance-reduced | Any | Rare events |
| **Bayesian Quadrature** | Optimal | Medium | Expensive functions |

## Interview Prep Checklist

✅ When to use Simpson's vs Trapezoidal  
✅ Why Gauss-Hermite beats Monte Carlo for Gaussian expectations  
✅ How to design importance sampling proposals  
✅ Change of variables formula for normalizing flows  
✅ EKF vs UKF vs Particle Filter trade-offs

## Industrial Applications Reviewed

- **NASA JPL**: Spacecraft trajectories
- **BlackRock**: Option pricing
- **Netflix**: Recommendations
- **JPMorgan**: Risk assessment
- **Tesla**: Autonomous driving
- **Spotify**: Music discovery

---

# Exercises

---

1. **Newton-Cotes**: Implement adaptive quadrature that recursively subdivides intervals where error exceeds tolerance.

2. **Gauss-Hermite**: Compute $\mathbb{E}[\sigma(X)]$ where $\sigma$ is the sigmoid function and $X \sim \mathcal{N}(0,1)$.

3. **Monte Carlo**: Implement stratified sampling and compare variance reduction to simple MC.

4. **Importance Sampling**: Design a proposal for estimating $P(X > 5)$ where $X \sim \mathcal{N}(0,1)$.

5. **Normalizing Flows**: Implement radial flows and compare expressiveness to planar flows.

6. **Time Series**: Implement a full UKF for a 2D tracking problem with nonlinear dynamics.

---

*Note: Solutions can be found in the `src/core/integration.py` and `src/core/normalizing_flows.py` modules.*